# ATIVIDADE EXTRA DE CÁLCULO NUMÉRICO

# Enunciado

Sejam p a raiz maior e q a raiz menor da equação
x^2+b*x-4=0.
Implemente a função que calcula essas raízes para dado b.

Para valores de p e q calculados considere o sistema

<p>2y+qz=-1</p>
<p>py-9z=3</p>

Determine se os métodos iterativos de Jacobi e Gauss-Seidel convergem para dado valor de b.

In [62]:
# PROGRAMA NA ÍNTEGRA, CONTENDO AS 3 FUNÇÕES ABORDADAS;

# Primeiro passo: Função para calcular as raízes p e q para dado b
# O cálculo foi realizado através do método iterativo de Newton-Raphson
import numpy as np # import necessário para trabalhar com diagonal da matriz envolvida

def calculaPQ():
    print("p e q são as raízes, maior e menor, respectivamente, da função x^2+b*x-4=0 para dado b\n")

    def funcao(x, b): # F(x)
        return x**2 + b*x -4 

    def derivada(x, b): # F'(x)
        return 2*x + b

    def NewtonRaphson(x): # Método Newton-Raphson para calcular x tal que F(x) = 0 para um dado erro
        return x - (funcao(x, b)/derivada(x, b)) # Xn+1 = Xn - (F(Xn)/F'(Xn))

    def iteracao(x, erro, count = 0): # Função que realiza as iterações
        x2 = NewtonRaphson(x)
        count += 1
        if (abs(x2-x) > erro):
            return iteracao(x2, erro, count)
        print("foram realizadas {} iterações para a precisão desejada".format(count))
        return x

    b = float(input("Digite o valor de b: \n")) # Exige valor de b
    casas = int(input("Digite o número de casas precisas depois da vírgula: \n")) # Casas depois da vírgula
    erro = 1*10**(-casas)

    if(b == 0): # Caso b seja 0, o chute inicial não pode ser b ou -b, pois haverá divisão por zero na F'(x)
        print("\nCálculo do p\n")
        p = iteracao(1.0, erro) # Positivo
        print("\nCálculo do q\n")
        q = iteracao(-1.0, erro) # Negativo
    elif(b > 0): 
        print("\nCálculo de p\n") 
        p = iteracao(0.1, erro) # Caso b seja MAIOR que 0, um bom chute inicial para p será um valor + próximo de 0
        print("\nCálculo de q\n")
        q = iteracao(-b, erro) # Caso b seja MAIOR que 0, um bom chute inicial será proporcional à -b para a raíz q
    else: 
        print("\nCálculo de p\n")
        p = iteracao(-b, erro) # Caso b seja MENOR que 0, um bom chute inicial para a raíz p será proporcional a -b
        print("\nCálculo de q\n")
        q = iteracao(-0.1, erro) # Caso b seja MENOR que 0, um bom chute inicial para q será um valor - próximo de 0

    print("\np = {}, q = {} erro em {}\n".format(round(p, casas), round(q, casas), erro))
    return p,q

# Segundo passo: Calculando y e z para resolver o sistema proposto através do método iterativo de Gauss-Siedel
# Sistem:
# 2y+qz=-1
# py-9z=3

# Gauss-Siedel

def calculaSistemaGaussSeidel(p, q):
    print("A partir das raízes p e q, será calculado y e z para o sistema pelo método de GAUSS-SEIDEL\n")
    print("Sistema: \nEq(1): 2y+qz=-1\nEq(2): py-9z=3\n")
    x = [0, 0] # Vetor inicial x                      
    a = [[2, q], [p, -9]] # Matriz do sistema a ser resolvido
    b = [-1,3] # Vetor b
    
    def verificaMatrizDiagonalDominante(a): # Verifica se a matriz do sistema em questão é diagonal dominante
        b = [[0,0], [0,0]]
        c = [[0,0], [0,0]]
        b[1][0],  b[1][1] = a[0][0], a[0][1]
        b[0][0],  b[0][1] = a[1][0], a[1][1]
        c[0][1],  c[1][1] = a[0][0], a[1][0]
        c[0][0],  c[1][0] = a[0][1], a[1][1]
        for i in range(len(a)): # Verificando diagonal dominante por linhas
            if(abs(a[i][i]) < abs(a[i][len(a)-1-i])):
                for j in range(len(b)):
                    if(abs(b[j][j]) < abs(b[j][len(b)-1-j])):
                        return False, a # Retorna que o sistema não é elegível de ser calculado
                    else:
                        return True, b # Retorna a matriz original, ou com as linhas trocadas
        for i in range(len(a)): # Verificando diagonal dominante por COLUNAS
            if(abs(a[i][i]) < abs(a[len(a)-1-i][i])):
                for j in range(len(c)):
                    if(abs(c[j][j]) < abs(c[len(c)-1-j][j])):
                        return False, a # Retorna que o sistema não é elegível de ser calculado
                    else:
                        return True, c # Retorna a matriz original, ou com as linhas trocadas
        return True, a
    
    def verificaSassenfeld(p,q): # Verifica se a matriz do sistema apresenta raio espectral < 1
        b1 = q/2
        b2 = (p/9)*b1
        if (b1 > b2):
            if (b1 < 1):
                return True
            else: 
                b1 = p/9 # Inverte as linhas e confere novamente
                b2 = (q/2)*b1
                if (b1 > b2):
                    if (b1 < 1):
                        return True
                    else:
                        return False
                else: 
                    if (b2 < 1):
                        return True
                    else:
                        return False
        if (b1 < b2):
            if (b2 < 1):
                return True
            else:
                b1 = p/9 # Inverte as linhas e confere novamente
                b2 = (q/2)*b1
                if (b1 > b2):
                    if (b1 < 1):
                        return True
                    else:
                        return False
                else: 
                    if (b2 < 1):
                        return True
                    else:
                        return False
        
    def verificaSolucaoSistema(y,z): # Função para verificar a convergência do método Gauss-Seidel a solução do sistema
        return (2*y + q*z + 1), (p*y - 9*z - 3) # As duas equações do sistema proposto.
    
    def Gauss_seidel(a, x ,b): # Método de Gauss-Seidel
        n = len(a)                
        for j in range(0, n):        
            d = b[j] # Variável temporária para estocar b[j]       
            for i in range(0, n): # Laço que realiza as operações    
                if(j != i):
                    d-=a[j][i] * x[i]       
            x[j] = d / a[j][j]          
        return x
    
    def iteracao(x, erro, count = 0): # Função que fará as iterações até chegar a precisão desejada
        x = Gauss_seidel(a, x, b)
        count +=1
        verifica = 2*x[0] + q*x[1] + 1
        verifica2 = p*x[0] - 9*x[1] - 3
        if(count > 30): # Limite para o número de iterações, caso seja atingido, retorna os últimos y,z calculados
            print("Limite de iterações atingido!\n")
            if (abs(verificaSolucaoSistema(x[0], x[1])) > 0.5):
                print("Não convergente!")
            return x
        print("{}, y = {}, z = {}. Substituindo Eq 1 e 2, resultado: (1) {}, (2) {}\n".format(
            count, round(x[0], casas), round(x[1], casas), 
            round(verificaSolucaoSistema(x[0],x[1])[0], casas), 
            round(verificaSolucaoSistema(x[0],x[1])[1], casas)))
        if (abs(verifica) > erro or abs(verifica2) > erro):
            return iteracao(x, erro, count)
        print("CONVERGENTE!\n")
        print("foram realizadas {} iterações para a precisão desejada\n".format(count))
        return x
    
    casas = int(input("Digite o número de casas precisas depois da vírgula: \n")) # Casas depois da vírgula
    erro = 1*10**(-casas)
    diagonalDominante, a = verificaMatrizDiagonalDominante(a) # Verifica a diagonal da matriz
    Sassenfeld = verificaSassenfeld(p,q) # Verifica se o critério de Sassenfeld é satisfeito
    if(diagonalDominante == False):
        print("A matriz do sistema não é diagonal dominante\n")
        print("Matriz:\n {}\n{}\n".format(a[0],a[1]))
        if (Sassenfeld == False):
            print("A matriz do sistema não satisfaz o critério de Sassenfeld!\n")
            print("Nenhum dos critérios de convergência foram satisfeitos!\n")
            print("Por garantia, o cálculo será executado até a 30ª iteração,\n")
            print("caso não converja para a solução, será retornado\n")
            print("'Não convergente' e mostrará que y,z obtidos não resultam\nnas raízes do sistema.\n")
        else:
            print("Porém apresenta o critério de Sassenfeld satisfeito.\n")
    x = iteracao(x, erro)
    print("y = {}, z = {} erro em {}\n".format(round(x[0], casas), round(x[1], casas), erro))
    print("CALCULADO POR GAUSS-SEIDEL")
    return x[0],x[1]

# Terceiro passo: Calculando y e z para resolver o sistema proposto através do método iterativo de Gauss-Jacobi
# Gauss-Jacobi

def calculaSistemaGaussJacobi(p, q):
    print("A partir das raízes p e q, será calculado y e z para o sistema pelo método de GAUSS-JACOBI\n")
    print("Sistema: \nEq(1): 2y+qz=-1\nEq(2): py-9z=3\n")
    x = [-0.50, -0.33] # Vetor para chute inicial x, -1/2 e -3/9                  
    a = [[2, q], [p, -9]] # Matriz do sistema a ser resolvido
    b = [-1,3] # Vetor b
    
    def verificaMatrizDiagonalDominante(a): # Verifica se a matriz do sistema em questão é diagonal dominante
        b = [[0,0], [0,0]]
        c = [[0,0], [0,0]]
        b[1][0],  b[1][1] = a[0][0], a[0][1]
        b[0][0],  b[0][1] = a[1][0], a[1][1]
        c[0][1],  c[1][1] = a[0][0], a[1][0]
        c[0][0],  c[1][0] = a[0][1], a[1][1]
        for i in range(len(a)): # Verificando diagonal dominante por linhas
            if(abs(a[i][i]) < abs(a[i][len(a)-1-i])):
                for j in range(len(b)):
                    if(abs(b[j][j]) < abs(b[j][len(b)-1-j])):
                        return False, a # Retorna que o sistema não é elegível de ser calculado
                    else:
                        return True, b # Retorna a matriz original, ou com as linhas trocadas
        for i in range(len(a)): # Verificando diagonal dominante por COLUNAS
            if(abs(a[i][i]) < abs(a[len(a)-1-i][i])):
                for j in range(len(c)):
                    if(abs(c[j][j]) < abs(c[len(c)-1-j][j])):
                        return False, a # Retorna que o sistema não é elegível de ser calculado
                    else:
                        return True, c # Retorna a matriz original, ou com as linhas trocadas
        return True, a
    
    def verificaSassenfeld(p,q): # Verifica se a matriz do sistema apresenta raio espectral < 1
        b1 = q/2
        b2 = (p/9)*b1
        if (b1 > b2):
            if (b1 < 1):
                return True
            else: 
                b1 = p/9 # Inverte as linhas e confere novamente
                b2 = (q/2)*b1
                if (b1 > b2):
                    if (b1 < 1):
                        return True
                    else:
                        return False
                else: 
                    if (b2 < 1):
                        return True
                    else:
                        return False
        if (b1 < b2):
            if (b2 < 1):
                return True
            else:
                b1 = p/9 # Inverte as linhas e confere novamente
                b2 = (q/2)*b1
                if (b1 > b2):
                    if (b1 < 1):
                        return True
                    else:
                        return False
                else: 
                    if (b2 < 1):
                        return True
                    else:
                        return False
    
    def verificaSolucaoSistema(y,z,q): # Função para verificar a convergência do método Gauss-Seidel a solução do sistema
        return (2*y + q*z + 1), (p*y - 9*z - 3) # As duas equações do sistema proposto.
    
    def Gauss_jacobi(a, b, x): # Método de Gauss-Jacobi para obtenção de solução para o sistema proposto
        # Create a vector of the diagonal elements of A                                                                                                                                                
        # and subtract them from A                                                                                                                                                                     
        D = np.diag(a) # separa a diagonal
        R = a - np.diagflat(D) # Cria uma matriz bidimensional, com a diagnoal dos valores inseridos (unidimensional)
        x = (b - np.dot(R,x))/D # Procedimento iterativo que será repetido, de acordo com a precisão desejada
        return x

    def iteracao(x, erro, count = 0): # Função que fará as iterações até chegar a precisão desejada
        x = Gauss_jacobi(a, b, x)
        count +=1
        verifica = 2*x[0] + q*x[1] + 1
        verifica2 = p*x[0] - 9*x[1] - 3
        if(count > 30): # Limite para o número de iterações, caso seja atingido, retorna os últimos y,z calculados
            print("Limite de iterações atingido!\n")
            if (abs(verificaSolucaoSistema(x[0], x[1])) > 0.5):
                print("Não convergente!")
            return x
        print("{}, y = {}, z = {}. Substituindo Eq 1 e 2, resultado: (1) {}, (2) {}\n".format(
            count, round(x[0], casas), round(x[1], casas), 
            round(verificaSolucaoSistema(x[0],x[1],q)[0], casas), 
            round(verificaSolucaoSistema(x[0],x[1],q)[1], casas)))
        if (abs(verifica) > erro or abs(verifica2) > erro):
            return iteracao(x, erro, count)
        print("CONVERGENTE!\n")
        print("foram realizadas {} iterações para a precisão desejada\n".format(count))
        return x
    
    casas = int(input("Digite o número de casas precisas depois da vírgula: \n")) # Casas depois da vírgula
    erro = 1*10**(-casas)
    diagonalDominante, a = verificaMatrizDiagonalDominante(a) # Verifica a diagonal da matriz
    Sassenfeld = verificaSassenfeld(p,q) # Verifica se o critério de Sassenfeld é satisfeito
    if(diagonalDominante == False):
        print("A matriz do sistema não é diagonal dominante\n")
        print("Matriz:\n {}\n{}\n".format(a[0],a[1]))
        if (Sassenfeld == False):
            print("A matriz do sistema não satisfaz o critério de Sassenfeld!\n")
            print("Nenhum dos critérios de convergência foram satisfeitos!\n")
            print("Por garantia, o cálculo será executado até a 30ª iteração,\n")
            print("caso não converja para a solução, será retornado\n")
            print("'Não convergente' e mostrará que y,z obtidos não resultam\nnas raízes do sistema.\n")
        else:
            print("Porém apresenta o critério de Sassenfeld satisfeito.\n")
    x = iteracao(x, erro)
    print("y = {}, z = {} erro em {}\n".format(round(x[0], casas), round(x[1], casas), erro))
    print("CALCULADO POR GAUSS-JACOBI")
    return x[0],x[1]

# Comandos para executar as funções

p, q = calculaPQ()
y, z = calculaSistemaGaussSeidel(p,q)
y, z = calculaSistemaGaussJacobi(p,q)

p e q são as raízes, maior e menor, respectivamente, da função x^2+b*x-4=0 para dado b

Digite o valor de b: 
69
Digite o número de casas precisas depois da vírgula: 
5

Cálculo de p

foram realizadas 3 iterações para a precisão desejada

Cálculo de q

foram realizadas 3 iterações para a precisão desejada

p = 0.05792, q = -69.05792 erro em 1e-05

A partir das raízes p e q, será calculado y e z para o sistema pelo método de GAUSS-SEIDEL

Sistema: 
Eq(1): 2y+qz=-1
Eq(2): py-9z=3

Digite o número de casas precisas depois da vírgula: 
5
A matriz do sistema não é diagonal dominante

Matriz:
 [2, -69.05792239128917]
[0.05792239126441791, -9]

Porém apresenta o critério de Sassenfeld satisfeito.

1, y = -0.5, z = -0.33655. Substituindo Eq 1 e 2, resultado: (1) 23.24153, (2) 0.0

2, y = -12.12076, z = -0.41134. Substituindo Eq 1 e 2, resultado: (1) 5.16478, (2) 0.0

3, y = -14.70316, z = -0.42796. Substituindo Eq 1 e 2, resultado: (1) 1.14773, (2) 0.0

4, y = -15.27702, z = -0.43165. Substitu

In [42]:
# Primeiro passo: Função para calcular as raízes p e q para dado b
# O cálculo foi realizado através do método iterativo de Newton-Raphson
import numpy as np # import necessário para trabalhar com diagonal da matriz envolvida

def calculaPQ():
    print("p e q são as raízes, maior e menor, respectivamente, da função x^2+b*x-4=0 para dado b\n")

    def funcao(x, b): # F(x)
        return x**2 + b*x -4 

    def derivada(x, b): # F'(x)
        return 2*x + b

    def NewtonRaphson(x): # Método Newton-Raphson para calcular x tal que F(x) = 0 para um dado erro
        return x - (funcao(x, b)/derivada(x, b)) # Xn+1 = Xn - (F(Xn)/F'(Xn))

    def iteracao(x, erro, count = 0): # Função que realiza as iterações
        x2 = NewtonRaphson(x)
        count += 1
        if (abs(x2-x) > erro):
            return iteracao(x2, erro, count)
        print("foram realizadas {} iterações para a precisão desejada".format(count))
        return x

    b = float(input("Digite o valor de b: \n")) # Exige valor de b
    casas = int(input("Digite o número de casas precisas depois da vírgula: \n")) # Casas depois da vírgula
    erro = 1*10**(-casas)

    if(b == 0): # Caso b seja 0, o chute inicial não pode ser b ou -b, pois haverá divisão por zero na F'(x)
        print("\nCálculo do p\n")
        p = iteracao(1.0, erro) # Positivo
        print("\nCálculo do q\n")
        q = iteracao(-1.0, erro) # Negativo
    elif(b > 0): 
        print("\nCálculo de p\n") 
        p = iteracao(0.1, erro) # Caso b seja MAIOR que 0, um bom chute inicial para p será um valor + próximo de 0
        print("\nCálculo de q\n")
        q = iteracao(-b, erro) # Caso b seja MAIOR que 0, um bom chute inicial será proporcional à -b para a raíz q
    else: 
        print("\nCálculo de p\n")
        p = iteracao(-b, erro) # Caso b seja MENOR que 0, um bom chute inicial para a raíz p será proporcional a -b
        print("\nCálculo de q\n")
        q = iteracao(-0.1, erro) # Caso b seja MENOR que 0, um bom chute inicial para q será um valor - próximo de 0

    print("\np = {}, q = {} erro em {}\n".format(round(p, casas), round(q, casas), erro))
    return p,q
p, q = calculaPQ()


p e q são as raízes, maior e menor, respectivamente, da função x^2+b*x-4=0 para dado b

Digite o valor de b: 
9
Digite o número de casas precisas depois da vírgula: 
4

Cálculo de p

foram realizadas 3 iterações para a precisão desejada

Cálculo de q

foram realizadas 3 iterações para a precisão desejada

p = 0.4244, q = -9.4245 erro em 0.0001



In [55]:
# Segundo passo: Calculando y e z para resolver o sistema proposto através do método iterativo de Gauss-Siedel
# Sistem:
# 2y+qz=-1
# py-9z=3

# Gauss-Siedel

def calculaSistemaGaussSeidel(p, q):
    print("A partir das raízes p e q, será calculado y e z para o sistema pelo método de Gauss-Seidel\n")
    print("Sistema: \nEq(1): 2y+qz=-1\nEq(2): py-9z=3\n")
    x = [0, 0] # Vetor inicial x                      
    a = [[2, q], [p, -9]] # Matriz do sistema a ser resolvido
    b = [-1,3] # Vetor b
    
    def verificaMatrizDiagonalDominante(a): # Verifica se a matriz do sistema em questão é diagonal dominante
        b = [[0,0], [0,0]]
        c = [[0,0], [0,0]]
        b[1][0],  b[1][1] = a[0][0], a[0][1]
        b[0][0],  b[0][1] = a[1][0], a[1][1]
        c[0][1],  c[1][1] = a[0][0], a[1][0]
        c[0][0],  c[1][0] = a[0][1], a[1][1]
        for i in range(len(a)): # Verificando diagonal dominante por linhas
            if(abs(a[i][i]) < abs(a[i][len(a)-1-i])):
                for j in range(len(b)):
                    if(abs(b[j][j]) < abs(b[j][len(b)-1-j])):
                        return False, a # Retorna que o sistema não é elegível de ser calculado
                    else:
                        return True, b # Retorna a matriz original, ou com as linhas trocadas
        for i in range(len(a)): # Verificando diagonal dominante por COLUNAS
            if(abs(a[i][i]) < abs(a[len(a)-1-i][i])):
                for j in range(len(c)):
                    if(abs(c[j][j]) < abs(c[len(c)-1-j][j])):
                        return False, a # Retorna que o sistema não é elegível de ser calculado
                    else:
                        return True, c # Retorna a matriz original, ou com as linhas trocadas
        return True, a
    
    def verificaSassenfeld(p,q): # Verifica se a matriz do sistema apresenta raio espectral < 1
        b1 = q/2
        b2 = (p/9)*b1
        if (b1 > b2):
            if (b1 < 1):
                return True
            else: 
                b1 = p/9 # Inverte as linhas e confere novamente
                b2 = (q/2)*b1
                if (b1 > b2):
                    if (b1 < 1):
                        return True
                    else:
                        return False
                else: 
                    if (b2 < 1):
                        return True
                    else:
                        return False
        if (b1 < b2):
            if (b2 < 1):
                return True
            else:
                b1 = p/9 # Inverte as linhas e confere novamente
                b2 = (q/2)*b1
                if (b1 > b2):
                    if (b1 < 1):
                        return True
                    else:
                        return False
                else: 
                    if (b2 < 1):
                        return True
                    else:
                        return False
        
    def verificaSolucaoSistema(y,z): # Função para verificar a convergência do método Gauss-Seidel a solução do sistema
        return (2*y + q*z + 1), (p*y - 9*z - 3) # As duas equações do sistema proposto.
    
    def Gauss_seidel(a, x ,b): # Método de Gauss-Seidel
        n = len(a)                
        for j in range(0, n):        
            d = b[j] # Variável temporária para estocar b[j]       
            for i in range(0, n): # Laço que realiza as operações    
                if(j != i):
                    d-=a[j][i] * x[i]       
            x[j] = d / a[j][j]          
        return x
    
    def iteracao(x, erro, count = 0): # Função que fará as iterações até chegar a precisão desejada
        x = Gauss_seidel(a, x, b)
        count +=1
        verifica = 2*x[0] + q*x[1] + 1
        verifica2 = p*x[0] - 9*x[1] - 3
        if(count > 30): # Limite para o número de iterações, caso seja atingido, retorna os últimos y,z calculados
            print("Limite de iterações atingido!\n")
            if (abs(verificaSolucaoSistema(x[0], x[1])) > 0.5):
                print("Não convergente!")
            return x
        print("{}, y = {}, z = {}. Substituindo Eq 1 e 2, resultado: (1) {}, (2) {}\n".format(
            count, round(x[0], casas), round(x[1], casas), 
            round(verificaSolucaoSistema(x[0],x[1])[0], casas), 
            round(verificaSolucaoSistema(x[0],x[1])[1], casas)))
        if (abs(verifica) > erro or abs(verifica2) > erro):
            return iteracao(x, erro, count)
        print("foram realizadas {} iterações para a precisão desejada\n".format(count))
        return x
    
    casas = int(input("Digite o número de casas precisas depois da vírgula: \n")) # Casas depois da vírgula
    erro = 1*10**(-casas)
    diagonalDominante, a = verificaMatrizDiagonalDominante(a) # Verifica a diagonal da matriz
    Sassenfeld = verificaSassenfeld(p,q) # Verifica se o raio espectral é menor que 1
    if(diagonalDominante == False):
        print("A matriz do sistema não é diagonal dominante\n")
        print("Matriz:\n {}\n{}\n".format(a[0],a[1]))
        if (Sassenfeld == False):
            print("A matriz do sistema não satisfaz o critério de Sassenfeld")
            print("O cálculo será executado até a 30ª iteração,\n")
            print("caso não converja para a solução, será retornado\n")
            print("'Não convergente' e mostrará que y,z obtidos não resultam\nnas raízes do sistema.\n")
        else:
            print("Porém apresenta o critério de Sassenfeld satisfeito.\n")
    x = iteracao(x, erro)
    print("y = {}, z = {} erro em {}\n".format(round(x[0], casas), round(x[1], casas), erro))
    return x[0],x[1]
y, z = calculaSistemaGaussSeidel(p,q)

A partir das raízes p e q, será calculado y e z para o sistema pelo método de Gauss-Seidel

Sistema: 
Eq(1): 2y+qz=-1
Eq(2): py-9z=3

Digite o número de casas precisas depois da vírgula: 
4
A matriz do sistema não é diagonal dominante

Matriz:
 [2, -9.424469413233458]
[0.4244421598383808, -9]

Porém apresenta o critério de Sassenfeld satisfeito.

1, y = -0.5, z = -0.3569. Substituindo Eq 1 e 2, resultado: (1) 3.3637, (2) 0.0

2, y = -2.1819, z = -0.4362. Substituindo Eq 1 e 2, resultado: (1) 0.7475, (2) 0.0

3, y = -2.5556, z = -0.4539. Substituindo Eq 1 e 2, resultado: (1) 0.1661, (2) 0.0

4, y = -2.6387, z = -0.4578. Substituindo Eq 1 e 2, resultado: (1) 0.0369, (2) 0.0

5, y = -2.6571, z = -0.4586. Substituindo Eq 1 e 2, resultado: (1) 0.0082, (2) 0.0

6, y = -2.6612, z = -0.4588. Substituindo Eq 1 e 2, resultado: (1) 0.0018, (2) -0.0

7, y = -2.6622, z = -0.4589. Substituindo Eq 1 e 2, resultado: (1) 0.0004, (2) 0.0

8, y = -2.6624, z = -0.4589. Substituindo Eq 1 e 2, resultado: (1

In [59]:
def calculaSistemaGaussJacobi(p, q):
    print("A partir das raízes p e q, será calculado y e z para o sistema pelo método de Gauss-Seidel\n")
    print("Sistema: \nEq(1): 2y+qz=-1\nEq(2): py-9z=3\n")
    x = [-0.50, -0.33] # Vetor para chute inicial x, -1/2 e -3/9                  
    a = [[2, q], [p, -9]] # Matriz do sistema a ser resolvido
    b = [-1,3] # Vetor b
    
    def verificaMatrizDiagonalDominante(a): # Verifica se a matriz do sistema em questão é diagonal dominante
        b = [[0,0], [0,0]]
        c = [[0,0], [0,0]]
        b[1][0],  b[1][1] = a[0][0], a[0][1]
        b[0][0],  b[0][1] = a[1][0], a[1][1]
        c[0][1],  c[1][1] = a[0][0], a[1][0]
        c[0][0],  c[1][0] = a[0][1], a[1][1]
        for i in range(len(a)): # Verificando diagonal dominante por linhas
            if(abs(a[i][i]) < abs(a[i][len(a)-1-i])):
                for j in range(len(b)):
                    if(abs(b[j][j]) < abs(b[j][len(b)-1-j])):
                        return False, a # Retorna que o sistema não é elegível de ser calculado
                    else:
                        return True, b # Retorna a matriz original, ou com as linhas trocadas
        for i in range(len(a)): # Verificando diagonal dominante por COLUNAS
            if(abs(a[i][i]) < abs(a[len(a)-1-i][i])):
                for j in range(len(c)):
                    if(abs(c[j][j]) < abs(c[len(c)-1-j][j])):
                        return False, a # Retorna que o sistema não é elegível de ser calculado
                    else:
                        return True, c # Retorna a matriz original, ou com as linhas trocadas
        return True, a
    
    def verificaSassenfeld(p,q): # Verifica se a matriz do sistema apresenta raio espectral < 1
        b1 = q/2
        b2 = (p/9)*b1
        if (b1 > b2):
            if (b1 < 1):
                return True
            else: 
                b1 = p/9 # Inverte as linhas e confere novamente
                b2 = (q/2)*b1
                if (b1 > b2):
                    if (b1 < 1):
                        return True
                    else:
                        return False
                else: 
                    if (b2 < 1):
                        return True
                    else:
                        return False
        if (b1 < b2):
            if (b2 < 1):
                return True
            else:
                b1 = p/9 # Inverte as linhas e confere novamente
                b2 = (q/2)*b1
                if (b1 > b2):
                    if (b1 < 1):
                        return True
                    else:
                        return False
                else: 
                    if (b2 < 1):
                        return True
                    else:
                        return False
    
    def verificaSolucaoSistema(y,z,q): # Função para verificar a convergência do método Gauss-Seidel a solução do sistema
        return (2*y + q*z + 1), (p*y - 9*z - 3) # As duas equações do sistema proposto.
    
    def Gauss_jacobi(a, b, x): # Método de Gauss-Jacobi para obtenção de solução para o sistema proposto
        # Create a vector of the diagonal elements of A                                                                                                                                                
        # and subtract them from A                                                                                                                                                                     
        D = np.diag(a) # separa a diagonal
        R = a - np.diagflat(D) # Cria uma matriz bidimensional, com a diagnoal dos valores inseridos (unidimensional)
        x = (b - np.dot(R,x))/D # Procedimento iterativo que será repetido, de acordo com a precisão desejada
        return x

    def iteracao(x, erro, count = 0): # Função que fará as iterações até chegar a precisão desejada
        x = Gauss_jacobi(a, b, x)
        count +=1
        verifica = 2*x[0] + q*x[1] + 1
        verifica2 = p*x[0] - 9*x[1] - 3
        if(count > 30): # Limite para o número de iterações, caso seja atingido, retorna os últimos y,z calculados
            print("Limite de iterações atingido!\n")
            if (abs(verificaSolucaoSistema(x[0], x[1])) > 0.5):
                print("Não convergente!")
            return x
        print("{}, y = {}, z = {}. Substituindo Eq 1 e 2, resultado: (1) {}, (2) {}\n".format(
            count, round(x[0], casas), round(x[1], casas), 
            round(verificaSolucaoSistema(x[0],x[1],q)[0], casas), 
            round(verificaSolucaoSistema(x[0],x[1],q)[1], casas)))
        if (abs(verifica) > erro or abs(verifica2) > erro):
            return iteracao(x, erro, count)
        print("foram realizadas {} iterações para a precisão desejada\n".format(count))
        return x
    
    casas = int(input("Digite o número de casas precisas depois da vírgula: \n")) # Casas depois da vírgula
    erro = 1*10**(-casas)
    diagonalDominante, a = verificaMatrizDiagonalDominante(a) # Verifica a diagonal da matriz
    Sassenfeld = verificaSassenfeld(p,q) # Verifica se o raio espectral é menor que 1
    if(diagonalDominante == False):
        print("A matriz do sistema não é diagonal dominante\n")
        print("Matriz:\n {}\n{}\n".format(a[0],a[1]))
        if (Sassenfeld == False):
            print("A matriz do sistema não satisfaz o critério de Sassenfeld")
            print("O cálculo será executado até a 30ª iteração,\n")
            print("caso não converja para a solução, será retornado\n")
            print("'Não convergente' e mostrará que y,z obtidos não resultam\nnas raízes do sistema.\n")
        else:
            print("Porém apresenta o critério de Sassenfeld satisfeito.\n")
    x = iteracao(x, erro)
    print("y = {}, z = {} erro em {}\n".format(round(x[0], casas), round(x[1], casas), erro))
    return x[0],x[1]

In [66]:
a = [[1,2], [3,4]]
b[1][0],  b[1][1] = a[0][0], a[0][1]
b[0][0],  b[0][1] = a[1][0], a[1][1]
c = True
for i in range(len(a)):
    if(abs(a[i][i]) < abs(a[i][len(a)-1-i])):
        for j in range(len(b)):
            if(abs(b[j][j]) < abs(b[j][len(b)-1-j])):
                c = False
                break
            else:
                a = b
a

[[1, 2], [3, 4]]

In [11]:
def verificaMatrizDiagonalDominante(a): # Verifica se a matriz do sistema em questão é elegível de ser calculado
        b = a
        c = a
        b[1][0],  b[1][1] = a[0][0], a[0][1]
        b[0][0],  b[0][1] = a[1][0], a[1][1]
        c[0][1],  c[1][1] = a[0][0], a[1][0]
        c[0][0],  c[1][0] = a[0][1], a[1][1]
        print(a,b,c)
        for i in range(len(a)): # Verificando diagonal dominante por linhas
            if(abs(a[i][i]) < abs(a[i][len(a)-1-i])):
                for j in range(len(b)):
                    if(abs(b[j][j]) < abs(b[j][len(b)-1-j])):
                        return False, a # Retorna que o sistema não é elegível de ser calculado
                    else:
                        return True, b # Retorna a matriz original, ou com as linhas trocadas
        for i in range(len(a)): # Verificando diagonal dominante por linhas
            if(abs(a[i][i]) < abs(a[len(a)-1-i][i])):
                for j in range(len(c)):
                    if(abs(c[j][j]) < abs(c[len(c)-1-j][j])):
                        return False, a # Retorna que o sistema não é elegível de ser calculado
                    else:
                        return True, c # Retorna a matriz original, ou com as linhas trocadas
        print(a,b,c)
        return True, a

In [12]:
A = [[1,2], [5,4]]
verificaMatrizDiagonalDominante(A)

[[1, 2], [1, 2]] [[1, 2], [1, 2]] [[1, 2], [1, 2]]


(False, [[1, 2], [1, 2]])

In [40]:
a = [[1,2], [5,4]]
c = [[0,0], [0,0]]
c[0][0],  c[1][0] = a[0][1], a[1][1]
c[0][1],  c[1][1] = a[0][0], a[1][0]
c

[[2, 1], [4, 5]]

In [27]:
a = [[1,2], [5,4]]
a[1][0]

5